<a href="https://colab.research.google.com/github/GrinddMaster/My_Grind/blob/main/GpuRuntime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report
import threading
from concurrent.futures import ThreadPoolExecutor

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/retinal-disease-classification/Labels.csv')
print(data.head())
images_path = '/content/drive/MyDrive/Colab Notebooks/retinal-disease-classification/Crs_val'

print(data.shape)
disease_counts = data.iloc[:, 2:].sum().sort_values(ascending=False)
fig = px.bar(disease_counts, title="Disease Distribution in Training Set")
fig.show()

threads = 10

In [ ]:
def load_images(img_id):#this function returns all the images and lables in one big array
  images = []
  for path in img_id:
    img_path = os.path.join(images_path, str(path)+".png")
    img = cv2.imread(img_path)
    if img is not None:
      image = cv2.resize(img, (246, 246))
      images.append(image)
    else:
        print(f"Failed to load image from path: {path}")
  return np.array(images)
def chunks_divide(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

data_chunk_size = len(data) // threads
data_chunks_list = list(chunks_divide(data["ID"], data_chunk_size))

with ThreadPoolExecutor(max_workers=threads) as executor:
    image_chunks = executor.map(load_images, data_chunks_list)

Xtrain = np.concatenate(list(image_chunks))
#Now all the iamges and Lables are in Xtrain and Ytrain.

In [ ]:
def load_lables(img_id):
  labels = []
  for label in img_id:
    row = data[data['ID'] == label].iloc[0]
    labels.append(row['Disease_Risk'])
  return np.array(labels)

def chunks_divide(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
label_chunk_size = len(data) // threads
label_chunks_list = list(chunks_divide(data["ID"], label_chunk_size))

with ThreadPoolExecutor(max_workers=threads) as executor:
    label_chunks = executor.map(load_lables, label_chunks_list)

Ytrain = np.concatenate(list(label_chunks))
print(Ytrain.shape)

In [ ]:
Xtrain = Xtrain/255.0 #Normalize the data.

In [ ]:
def Model_Training():
  model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(246, 246, 3)),
      MaxPooling2D((2, 2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Conv2D(128, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Flatten(),
      Dense(46, activation='relu'),
      Dense(50, activation='relu'),
      Dense(182, activation='relu'),
      Dense(40, activation='relu'),
      Dropout(0.5),
      Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = Model_Training()

kf = KFold(n_splits=20)
fold_no = 1
X = Xtrain.copy()
Y = Ytrain.copy()
for train_index, val_index in kf.split(X):
    Xtrain_fold,Xval_fold = X[train_index],X[val_index]
    Ytrain_fold,Yval_fold = Y[train_index],Y[val_index]
    print(f'Training fold {fold_no}...')

    history=model.fit(Xtrain_fold, Ytrain_fold, epochs=10, validation_data=(Xval_fold, Yval_fold))

    fold_no += 1


In [ ]:

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()